# 5 start monitoring & logging on minikube

change ${PJ_ROOT} to your directory.

In [ ]:
export PJ_ROOT="${HOME}/core"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/core
```

## load environment variables

In [ ]:
source ${PJ_ROOT}/docs/environments/minikube/env

## start fiware cygnus for elasticsearch

In [ ]:
kubectl apply -f cygnus/cygnus-elasticsearch-service.yaml

In [ ]:
kubectl apply -f cygnus/cygnus-elasticsearch-deployment.yaml

In [ ]:
kubectl get pods -l app=cygnus-elasticsearch

example)
```
NAME                                   READY     STATUS    RESTARTS   AGE
cygnus-elasticsearch-69f5cd7dc-766pp   1/1     Running   0          109s
cygnus-elasticsearch-69f5cd7dc-crw8w   1/1     Running   0          109s
cygnus-elasticsearch-69f5cd7dc-d9ggb   1/1     Running   0          109s
```

In [ ]:
kubectl get services -l app=cygnus-elasticsearch

example)
```
NAME                    TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)             AGE
cygnus-elasticsearch    ClusterIP   10.103.255.240   <none>        5050/TCP,8081/TCP   1m
```

## start prometheus & grafana

### install coreos/prometheus-operator

In [ ]:
helm install stable/prometheus-operator --name po --namespace monitoring -f monitoring/prometheus-operator-minikube.yaml

In [ ]:
kubectl --namespace monitoring get pods -l "app=prometheus-operator-operator,release=po"

example)
```
NAME                                               READY   STATUS    RESTARTS   AGE
po-prometheus-operator-operator-7cf7c5cc97-78h9g   1/1     Running   0          2m28s
```

In [ ]:
kubectl get daemonsets --namespace monitoring

example)
```
NAME                          DESIRED   CURRENT   READY   UP-TO-DATE   AVAILABLE   NODE SELECTOR   AGE
po-prometheus-node-exporter   1         1         1       1            1           <none>          3m47s
```

In [ ]:
kubectl get deployments --namespace monitoring

example)
```
NAME                              READY   UP-TO-DATE   AVAILABLE   AGE
po-grafana                        1/1     1            1           4m27s
po-kube-state-metrics             1/1     1            1           4m27s
po-prometheus-operator-operator   1/1     1            1           4m27s
```

In [ ]:
kubectl get statefulsets --namespace monitoring

example)
```
NAME                                               READY   AGE
alertmanager-po-prometheus-operator-alertmanager   1/1     4m42s
prometheus-po-prometheus-operator-prometheus       1/1     4m32s
```

In [ ]:
kubectl get pods --namespace monitoring

example)
```
NAME                                                 READY   STATUS    RESTARTS   AGE
alertmanager-po-prometheus-operator-alertmanager-0   2/2     Running   0          5m10s
po-grafana-594865f85-rqdcs                           2/2     Running   0          5m39s
po-kube-state-metrics-844bd6548c-xdzb4               1/1     Running   0          5m39s
po-prometheus-node-exporter-9gpwj                    1/1     Running   0          5m39s
po-prometheus-operator-operator-6984bb6db6-8cwft     1/1     Running   0          5m39s
prometheus-po-prometheus-operator-prometheus-0       3/3     Running   1          5m
```

In [ ]:
kubectl get services --namespace monitoring

example)
```
NAME                                  TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)             AGE
alertmanager-operated                 ClusterIP   None             <none>        9093/TCP,6783/TCP   5m46s
po-grafana                            ClusterIP   10.106.85.127    <none>        80/TCP              6m16s
po-kube-state-metrics                 ClusterIP   10.106.251.12    <none>        8080/TCP            6m16s
po-prometheus-node-exporter           ClusterIP   10.97.49.161     <none>        9100/TCP            6m16s
po-prometheus-operator-alertmanager   ClusterIP   10.97.209.186    <none>        9093/TCP            6m16s
po-prometheus-operator-operator       ClusterIP   10.108.31.165    <none>        8080/TCP            6m15s
po-prometheus-operator-prometheus     ClusterIP   10.101.223.201   <none>        9090/TCP            6m15s
prometheus-operated                   ClusterIP   None             <none>        9090/TCP            5m36s
```

### edit some prometheus rules

In [ ]:
echo 'kubectl edit --namespace=monitoring prometheusrules po-prometheus-operator-general.rules'

```diff
       for: 10m
       labels:
         severity: warning
-    - alert: Watchdog
-      annotations:
-        message: |
-          This is an alert meant to ensure that the entire alerting pipeline is functional.
-          This alert is always firing, therefore it should always be firing in Alertmanager
-          and always fire against a receiver. There are integrations with various notification
-          mechanisms that send a notification when this alert is not firing. For example the
-          "DeadMansSnitch" integration in PagerDuty.
-      expr: vector(1)
-      labels:
-        severity: none
```

### confirm prometheus

In [ ]:
echo 'kubectl --namespace monitoring port-forward $(kubectl get pod --namespace monitoring -l prometheus=kube-prometheus -l app=prometheus -o template --template "{{(index .items 0).metadata.name}}") 9090:9090'

#### for macOS

In [ ]:
open http://localhost:9090

#### for Ubuntu

In [ ]:
xdg-open http://localhost:9090

1. confirm Prometheus
    * no `Target` is down.
    * no `Alert` is fired except CPU or Memory resources.

### setup grafana

In [ ]:
echo 'kubectl --namespace monitoring port-forward $(kubectl get pod --namespace monitoring -l app=grafana,release=po -o template --template "{{(index .items 0).metadata.name}}") 3000:3000'

#### for macOS

In [ ]:
open http://localhost:3000/login

#### for Ubuntu

In [ ]:
xdg-open http://localhost:3000/login

1. login grafana
    * At the first, a admin user (`admin`/`prom-operator`) is available.
2. change the admin's password

## start Elasticsearch, fluentd and Kibana

### start Elasticsearch

In [ ]:
kubectl apply -f logging/elasticsearch-minikube-service.yaml

In [ ]:
kubectl apply -f logging/elasticsearch-minikube-deployment.yaml

In [ ]:
kubectl get statefulsets --namespace monitoring -l k8s-app=elasticsearch-logging

example)
```
NAME                    READY   AGE
elasticsearch-logging   2/2     3m20s
```

In [ ]:
kubectl get pods --namespace monitoring -l k8s-app=elasticsearch-logging

example)
```
NAME                      READY   STATUS    RESTARTS   AGE
elasticsearch-logging-0   1/1     Running   0          3m30s
elasticsearch-logging-1   1/1     Running   0          2m27s
```

In [ ]:
kubectl get services --namespace monitoring -l k8s-app=elasticsearch-logging

example)
```
NAME                    TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)    AGE
elasticsearch-logging   ClusterIP   10.100.232.146   <none>        9200/TCP   3m40s
```

In [ ]:
kubectl exec -it elasticsearch-logging-0 --namespace monitoring -- curl -H "Content-Type: application/json" -X PUT http://elasticsearch-logging:9200/_cluster/settings -d '{"transient": {"cluster.routing.allocation.enable":"all"}}'

### start fluentd

In [ ]:
kubectl apply -f logging/fluentd-es-configmap.yaml

In [ ]:
kubectl apply -f logging/fluentd-es-ds.yaml

In [ ]:
kubectl get daemonsets --namespace monitoring -l k8s-app=fluentd-es

example)
```
NAME                DESIRED   CURRENT   READY   UP-TO-DATE   AVAILABLE   NODE SELECTOR   AGE
fluentd-es-v2.4.0   1         1         1       1            1           <none>          88s
```

In [ ]:
kubectl get pods --namespace monitoring -l k8s-app=fluentd-es

example)
```
NAME                      READY   STATUS    RESTARTS   AGE
fluentd-es-v2.4.0-ltcxj   1/1     Running   0          2m48s
```

### start Kibana

In [ ]:
kubectl apply -f logging/kibana-service.yaml

In [ ]:
kubectl apply -f logging/kibana-deployment.yaml

In [ ]:
kubectl get pods --namespace monitoring -l k8s-app=kibana-logging

example)
```
NAME                              READY     STATUS    RESTARTS   AGE
kibana-logging-7fd755b6dc-4x8ln   1/1       Running   0          40s
```

### start curator

In [ ]:
kubectl apply -f logging/curator-configmap.yaml

In [ ]:
kubectl apply -f logging/curator-cronjob.yaml

In [ ]:
kubectl get cronjobs --namespace monitoring -l k8s-app=elasticsearch-curator

example)
```
NAME                    SCHEDULE     SUSPEND   ACTIVE    LAST SCHEDULE   AGE
elasticsearch-curator   0 18 * * *   False     0         <none>          9s
```

### setup Kibana

In [ ]:
echo 'kubectl --namespace monitoring port-forward $(kubectl get pod -l k8s-app=kibana-logging --namespace monitoring -o template --template "{{(index .items 0).metadata.name}}") 5601:5601'

#### for macOS

In [ ]:
open http://localhost:5601/

#### for Ubuntu

In [ ]:
xdg-open http://localhost:5601/

1. show `Management -> Index Patterns`
2. set `logstash-*` as Index Pattern, and push `Next step`
3. set `@timestamp` as Time Filter field name, and push `Create index pattern`

### setup grafana

In [ ]:
echo 'kubectl --namespace monitoring port-forward $(kubectl get pod --namespace monitoring -l app=grafana,release=po -o template --template "{{(index .items 0).metadata.name}}") 3000:3000'

#### for macOS

In [ ]:
open http://localhost:3000/login

#### for Ubuntu

In [ ]:
xdg-open http://localhost:3000/login

### add `elasticsearch` dashboard to grafana

1. add a new Data Source (ElasticSearch)
    * Name: `elasticsearch`
    * URL: `http://elasticsearch-logging:9200`
    * Access: `Server(Default)`
    * Index name: `logstash-*`
    * Time field name: `@timestamp`
    * Version: `6.0+`
2. import `monitoring/dashboard_elasticsearch.json`